## STEP  FIVE OF PROCESSING PIPELINE

#### This script imports original CRF and No SuperSat Data and computes the Naka Rushton / Hyperbolic ratio parameters via SciPy.curve_fit for both data sets.These parameters are saved and then exported as dict file for ANOVA and visual inspection (dict -> .pkl file)

#### Pending goals ...
#### 1|Save data as a .mat file to compare with martins code 

In [ ]:
# load packages
import numpy as np 
from scipy.io   import  loadmat
import pandas as pd
import os
import matplotlib.pyplot as plt #import matplotlib as plt
from scipy.optimize import curve_fit 
import seaborn as sns #import mat73
import pickle as pkl
from datetime import datetime

In [ ]:
NumBins = 6 # number of contrasts
NumHarms = 2 # number of harmonic data: 2F1, 4F1
NumComp = 0 # first component from RCA
NumConds = 4

In [ ]:
cp1 = sns.color_palette("cubehelix",12) 
cp2 = sns.color_palette("hls", 12)

In [ ]:
# Main Directory of processed file from MatLab
# MainDir = 'D:\\AttnXV3_analysis\\RCA_F1\\AvgCRFs\\' # set dir
MainDir = 'c:\\plimon\\LTP_analysis\\RCA_F1\\AvgCRFs\\' # set dir
os.chdir(MainDir) # change old dir, to this dir
d = os.listdir(MainDir) # list files in dir
print(f'Files on hand: {d}')
##############################################
FileN = d[0] # choose one                        
file_path1 = os.path.join(MainDir, FileN) # join paths and prep 2 load
print('Current WD:',file_path1) # does path exist ... ?
print('Does File #1 Exist?',os.path.exists(file_path1)) # yes or no

In [ ]:
loadData = pkl.load(open(file_path1,'rb'))
print(loadData.keys())

In [ ]:
data = loadData['Data']
data_labels = loadData['DictMainKeys']
SubNames = np.array(loadData['SubNames'])
note = loadData['VoiceMemo']
contrast_levels = loadData['ContLevs']
contrast_labs = loadData['crfLabs']
print(note)
NumSubs = int(len(SubNames))
print(data_labels) # labels for nr plots

## Naka Rushton Model Function

In [ ]:
def naka_rushton(C, C50, n, Rmax, b):
    """
    This function models neural responses captured by Contrast Response Functions
     
    C50 (parameter 1) : Contrast level that yields half of max response (contrast %)
    n (parameter 2)   : Determines the curve arc for fitting (small integer)
    Rmax (Parameter 3): Max response elicited (mv)
    b (Parameter 4)   : Starting value (origin)
    """
    return ((Rmax * ((C**n) / (C50**n + C**n)))+b)

#### Other Important Functions ...

In [ ]:
def ReshapeData(NumConds,NumBins,x):
    """
    Reshape CRF data for all conditions to feed into scipy optimally
    12 x 2 -> 4x6 array for scipy curve fit
    """
    dictOut = {}
    NumSubs = int(len(x.keys())) # 25
    for i in (range(NumSubs)):
        yOut = np.zeros((NumConds,NumBins)) # how data is going to be stored
        yIn = x[i]
        yOut[0,:] = yIn[:NumBins,0] # 2f Pre
        yOut[1,:] = yIn[:NumBins,1] # 2f Post
        yOut[2,:] = yIn[NumBins:,0] # 4f Pre
        yOut[3,:] = yIn[NumBins:,1] # 4f Post
        dictOut[i] = np.array(yOut)
    return dictOut
#######################################################################################################
def FixSuperSat(DictIn,NumBins,NumConds,NumSubs):
    """
    Fix Supersaturation at higher contrasts by setting proceeding vals to max val
    before supersaturation occurs, basically lift drooping *tail*
    DictIn = Dict key 
    """
    dictOut = {}
    for i in range(NumSubs):
        aIn = DictIn[i] # 4x6
        aOut = np.zeros_like(aIn)
        max_pos, max_val = np.argmax(aIn,axis=1), np.max(aIn,axis=1) # 4 array 
        for co in range(NumConds):
            pos = max_pos[co]
            amp = max_val[co]
            if pos < NumBins:
                aOut[co,:pos] = aIn[co,:pos]
                aOut[co,pos:] = amp
            else:
                aOut[co,:] = aIn[co,:]
        dictOut[i] = aOut
    return dictOut
#######################################################################################################
def HyperRatComputation(NumConds,NumSubs,naka_rushton,contrast_levels,init_g1,init_g2,boundaries,xDict):
    """
    1st pass - Run Curve Fit for each condition for 1 subject at the same time and save params into dict
    Goal : save cov to plot error bars!!!
    """
    nr_data = {}
    for subj in range(NumSubs):
        dIn = xDict[subj]
        nr_out = np.zeros((NumConds,NumConds)) # 4 x 4 array
        for cond in range(NumConds):
            #if c<=2:
            if cond<=1:
                # NR for 2F harmonic pre and post
                p, cov = curve_fit(naka_rushton,contrast_levels,dIn[cond,:],p0 = init_g1, bounds = boundaries, nan_policy = 'omit', method = 'trf' )
            else:
                # NR for 4F harmonic pre and post
                p, cov = curve_fit(naka_rushton,contrast_levels,dIn[cond,:],p0 = init_g2, bounds = boundaries, nan_policy = 'omit', method = 'trf')
            nr_out[cond,:] = p
        nr_data[subj] = np.array(nr_out)
    return nr_data
#######################################################################################################
def SmoothHyperRat(naka_rushton,NumConds,NumSubs,ContrastEnd,Full_Contrasts,dict_In):# = HyperBolicParams[0]):
    """
    2nd pass - Run Naka Rushton Model *again* for all contrasts 1% - 100% for all conditions to get a smoother fit of the data 
    returns data dict
    """
    dictOut = {}
    for i in range(NumSubs):
        xIn = dict_In[i] # 4x4 array 
        xOut = np.zeros((NumConds,ContrastEnd)) # 
        for co in range(NumConds):                   # c50   |   n    |  Rmax  |    b
            hyperRat = naka_rushton(Full_Contrasts,xIn[co,0],xIn[co,1],xIn[co,2],xIn[co,3])
            xOut[co,:] = hyperRat
        dictOut[i] = xOut
    return dictOut
#######################################################################################################
def GenImgs(NumConds,contrast_levels,x2,sIn,CRF_In,NRParams_In):
#sIn = 4
#CRF_In = pyData_clean
#NRParams_In = RefinedHyperBolicParams
    """
    Plot Everyones CRF and NR Model produced by SmoothHyperRat,
    serves as a sanity check to see if the model captured that data appropriately / didnt get stuck on a boundary
    """
    fig,axs = plt.subplots(2,4, figsize = (16,8),sharey=True)
    for files in range(NumConds):
        x_CRF = CRF_In[files][sIn]
        x_nr_params = NRParams_In[files][sIn]
        for co in range(NumConds):
            axs[0,files].plot(x2,x_nr_params[0,:],color = cp1[3], label = '2f pre')
            axs[0,files].plot(x2,x_nr_params[1,:],color = cp1[7], label = '2f post')

            axs[0,files].scatter(contrast_levels,x_CRF[0,:],color = cp1[3],alpha = 0.5, label = '2f pre')
            axs[0,files].scatter(contrast_levels,x_CRF[1,:],color = cp1[7],alpha = 0.5, label = '2f pre')

            axs[0,files].set_title(f'{data_labels[files]}')
            #########################################################################
            if files == 0 and co == 0:
                axs[0,files].legend(loc = 'lower right')
            axs[1,files].plot(x2,x_nr_params[2,:],color = cp1[3], label = '4f pre')
            axs[1,files].plot(x2,x_nr_params[3,:],color = cp1[7], label = '4f post')
                
            axs[1,files].scatter(contrast_levels,x_CRF[2,:],color = cp1[3],alpha = 0.5, label = '4f pre')
            axs[1,files].scatter(contrast_levels,x_CRF[3,:],color = cp1[7],alpha = 0.5, label = '4f pre')
    return fig, axs

#### Reshape Data to Feed into SciPy

In [ ]:
pyData = {} # reshaped data to enter into curve fit
for co in range(NumConds):
    pyData[co] = ReshapeData(NumConds,NumBins,x=data[co])

#### Remove SuperSaturation

In [ ]:
pyData_clean = {}
for co in range(NumConds):
    pyData_clean[co] = FixSuperSat(pyData[co],NumBins,NumConds,NumSubs)

In [ ]:
# sanity check for everything thats been done so far ...
sns.set_theme()
fig,axs = plt.subplots(1,2, figsize = (3,1.5),sharey=True)
#############    Compare Here    #################
d = pyData_clean[2] # no super saturation CRF's  #
#d = pyData[2] # original data CRF               #
##################################################
a = np.array(d[3]) # 4x 6 
for i in range(NumConds):
    axs[0].plot(a[0,:],label = 'pre', color =  cp1[6])
    axs[0].plot(a[1,:],label='post', color = cp1[3])
    if i  == 0:
        axs[0].legend(loc = 'lower right', fontsize = 8)

    axs[1].plot(a[2,:],label = 'pre', color =  cp1[6])
    axs[1].plot(a[3,:],label='post', color = cp1[3])

#### Set Boundaries and Initial Est

In [ ]:
### Model Estimates...based on nothing soild - subject to change 
c50_b = [0,300]
n_b = [0,5] # [0,10]
Rmax_b = [0,10] # [0,10]
b_b = [0,3]
boundaries=([c50_b[0],n_b[0],Rmax_b[0],b_b[0]], [c50_b[1],n_b[1],Rmax_b[1],b_b[1]]) # Boundary Params for C50, N, Rmax and B
init_g1 = [20,3,4,0.5] # Initial guess for 2f params
init_g2 = [10,1,2,0.5] # Initial guess for 4f params

param_names=['C50','n','Rmax','b']

#### Run Curve Fit

In [ ]:
HyperBolicParams = {}
for i in range(NumConds):
    HyperBolicParams[i] = HyperRatComputation(NumConds,NumSubs,naka_rushton,contrast_levels, init_g1,init_g2, boundaries, xDict = pyData_clean[i])

#### Re-Run Hyperbolic Ratio Params Into HyperRat for a Smoother Fit

In [ ]:
Full_Contrasts = np.array(np.arange(0.1,100,0.1)) # all contrast from 1% - 100% - ~1k data points
ContrastEnd = (Full_Contrasts.shape[0])
print('length of x array for contrasts: ', Full_Contrasts.shape)

x1 = np.linspace(0, 100, NumBins) # all 6 contrasts
x2 = np.linspace(0, 100, ContrastEnd) # all 100 contrasts

In [ ]:
RefinedHyperBolicParams = {}

for i in range(NumConds):
    RefinedHyperBolicParams[i] = SmoothHyperRat(naka_rushton, NumConds,NumSubs,ContrastEnd,Full_Contrasts, dict_In = HyperBolicParams[i])

### Plot Some Figs ...

In [ ]:
for su in range(4):#(NumSubs):
    skdjv,x = GenImgs(NumConds,contrast_levels,x2,sIn = su, CRF_In = pyData_clean, NRParams_In = RefinedHyperBolicParams)

#### Save Data!

In [ ]:
# set save data dir 
# Set directory to save NR Data in ..new folder
# SaveDataDir = 'D:\\AttnXV3_analysis\\RCA_F1\\HyperRatioParams\\' # set dir where files (.pkl, .csv) will be saved
SaveDataDir = 'C:\\plimon\\LTP_analysis\\RCA_F1\\HyperRatioParams\\' # set dir where files (.pkl, .csv) will be saved
FileOutName = 'All_HRP_Data_FixedSupSat_CRFuncs' # make sure this file changes each time you save
######################################################
dnt = datetime.now() # add date and time bc im wreckless when saving ..
fdnt = dnt.strftime("%Y%m%d_%H%M") # set the above as a string ...
FileN = f'{FileOutName}_{fdnt}.pkl' 
#MatLabFileN = f'{FileOutName}_{fdnt}.mat'

NewFileNPath = os.path.join(SaveDataDir,FileN)
#Mat_NewFileNPath = os.path.join(SaveDataDir, MatLabFileN)

print('Full New File Dir: ', NewFileNPath)
# #print('Where matlab file will be stored',Mat_NewFileNPath )

if not os.path.exists(SaveDataDir):
     os.makedirs(SaveDataDir)
     print('Path to Save File is:',SaveDataDir)

In [ ]:
# add another personal voice memo ...
Note = ['This file contains all subjects who did both sessions (fixed super saturation), RN Params, and model based on NR params for all conditions']

DictOut = {}

DictOut['nr_params'] = HyperBolicParams # 4 x 4 array of values per condition 
DictOut['HyperRatioModels'] = RefinedHyperBolicParams # smoother version of nr_params
DictOut['SubNames'] = SubNames # subject names
DictOut['DictMainKeys'] = data_labels # conditions across all 4 files .. 
DictOut['ContLevs'] = contrast_levels # expt contrasts used
DictOut['crfLabs'] = contrast_labs # same as above but string
DictOut['VoiceMemo'] = Note # personal note in case i forget
DictOut['plot_nr_arr'] = x2 # array to plot Hyperratiomodels model for 1-100% contrast
# Info about scipy params
DictOut['model_bounds'] = boundaries # bounds fed to scipy
DictOut['param_order'] = param_names # order in which params were stored
DictOut['2f_initalGuess'] = init_g1 # initial guess for model
DictOut['4f_initalGuess'] = init_g2 # initial guess for model

In [ ]:
print(NewFileNPath)

In [ ]:
saveFile = 'n'

if saveFile == 'y':
 
 with open(NewFileNPath, 'wb') as file:
    pkl.dump(DictOut, file, protocol=pkl.HIGHEST_PROTOCOL)
    # save as .mat file or .csv file to import into matlab 
    
    print('Data Saved! :))')
else:
    print('Did Not Save File! Change file name before switching to y!')